<a href="https://colab.research.google.com/github/yimstar9/Engineer-Big-Data-Analysis_R/blob/main/%EA%B3%B5%EC%8B%9D_%EC%98%88%EC%8B%9C_%EB%AC%B8%EC%A0%9C_%EC%9E%91%EC%97%85%ED%98%952.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#https://www.kaggle.com/code/agileteam/t2-exercise-tutorial-baseline/notebook
# 백화점 고객의 1년 간 구매 데이터를 활용해
# 데이터 전처리
# Feature Engineering
# 모델링 (분류 알고리즘 사용)
# 하이퍼파라미터 튜닝 (초매개변수 최적화)
# 모형 앙상블
# csv제출
# 유의사항
# 수험번호.csv 파일이 만들어지도록 코드를 제출함
# 제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점함
# 데이터 출처 및 연결
# data 출처: https://www.dataq.or.kr/ - 공지사항 - 제2회 빅데이터분석기사 실기 안내 - 첨부파일
# 데이터셋 업로드
# 데이터셋 프라이빗 업로드 : https://youtu.be/BZlEQ5JwLiA
# Datasets - new dataset - (drag&drop) - Create / 반드시 Private
# 작업형2 예시: https://youtu.be/_GIBVt5-khk

library(caret)
library(dplyr)
library(randomForest)
library(ModelMetrics)
x_train<-read.csv("X_train.csv",fileEncoding = 'euc-kr', encoding = 'utf-8')
y_train<-read.csv("y_train.csv")
x_test<-read.csv("X_test.csv",fileEncoding = 'euc-kr', encoding = 'utf-8')


# 사용자 코딩
colSums(is.na(x_train))
x_train$환불금액<-ifelse(is.na(x_train$환불금액),0,x_train$환불금액)
x_train$총구매액<-ifelse(x_train$총구매액<0,0,x_train$총구매액)
x_train$최대구매액<-ifelse(x_train$최대구매액<0,0,x_train$최대구매액)

s<-preProcess(x_train,"range")
x_train[,-c(1,5,6,9)]<-predict(s,x_train)[,-c(1,5,6,9)]

x_train$주구매상품<-as.factor(x_train$주구매상품)
x_train$주구매지점<-as.factor(x_train$주구매지점)

x_test$주구매상품<-as.factor(x_test$주구매상품)
x_test$주구매지점<-as.factor(x_test$주구매지점)
x_test$환불금액<-ifelse(is.na(x_test$환불금액),0,x_test$환불금액)
levels(x_test$주구매상품)<-levels(x_train$주구매상품)

y_train$gender<-as.factor(y_train$gender)



df<-inner_join(x_train,y_train,by='cust_id')
idx<-sample(1:nrow(df),0.7*nrow(df))
train_df<-df[idx,]
test_df<-df[-idx,]

m1_rf<-randomForest(gender~.,train_df[,-c(1,7,10)],ntree=100)
pred<-predict(m1_rf,test_df[,-c(1,7,10)],type="response")

print(caret::confusionMatrix(test_df$gender, pred)$overall[1])
print(paste0("auc:",auc(test_df$gender, pred)))


m_rf<-randomForest(gender~.,df[,-c(1,7,10)],ntree=100)
ans<-predict(m_rf,x_test[,-c(1,7,10)],type="prob")


result<-cbind(x_test[,1],as.character(ans[,2]))
colnames(result)<-c("cust_id","gender")

write.csv(result,"0099.csv",row.names=F)
zz<-read.csv("0099.csv")
head(zz)




# 답안 제출 참고
# 아래 코드 변수명과 수험번호를 개인별로 변경하여 활용
# write.csv(변수명,'003000000.csv',row.names=F) 
